In [ ]:
from pyspark.sql import SparkSession
# Create Spark session
spark = SparkSession.builder.appName("MyApp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/03 11:42:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# SparkUI
print(spark.sparkContext.uiWebUrl)

http://bagjongseon-ui-Macmini.local:4040


In [ ]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("../data/retail-data/by-day/*.csv")

staticDataFrame.createOrReplaceTempView("retail_data")

In [7]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



수치형 변환
- 머신러닝 알고리즘을 적용하기 위해서는 수치형 데이터가 필요
- 트랜스포메이션을 사용해 날짜 데이터를 수치형 데이터로 변환

In [9]:
from pyspark.sql.functions import date_format, col

preppedDataFrame = staticDataFrame\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
  .coalesce(5)

1. Fill missing values with 0 → `.na.fill(0)`
2. Extract day of the week from InvoiceDate → `.withColumn("day_of_week", date_format(...))`
3. Reduce partitions to 5 for efficiency → `.coalesce(5)`

데이터를 학습셋과 테스트셋으로 분리
- 시계열 데이터
- 특정 날짜(2011-07-11)를 기준으로 직접 분리
  - 이전: 학습셋
  - 이후: 테스트 데이터셋

In [10]:
trainDataFrame = preppedDataFrame\
  .where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame\
  .where("InvoiceDate >= '2011-07-01'")

액션을 호출해 데이터를 분리
- 임의 날짜를 기준으로 데이터를 분리
- 예제의 코드는 데이터셋을 대략 절반으로 나눔

In [12]:
trainDataFrame.count()

245903

In [13]:
testDataFrame.count()

296006

In [ ]:
`StringIndexer`
- StringIndexer is a feature transformer in PySpark that converts categorical string values into numerical indices. 
- It is commonly used in machine learning pipelines, where models require numerical input.

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer()\
  .setInputCol("day_of_week")\
  .setOutputCol("day_of_week_index")

`day_of_week` (요일)을 수치형으로 반환
- 토요일 -> 6
- 월요일 -> 1

문제점: 토요일이 월요일 보다 크다고 인식 할 수 있다.

`OneHotEncoder`를 사용해 각 값을 자체 컬럼으로 인코딩해야 한다.
- 특정 요일이 해당 요일인지 아닌지 불리언 타입으로 나타낼 수 있다.


In [15]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder()\
  .setInputCol("day_of_week_index")\
  .setOutputCol("day_of_week_encoded")

벡터 타입을 구성할 컬럼 중 하나로 사용된다.

In [17]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler()\
  .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
  .setOutputCol("features")

In [18]:
from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
  .setStages([indexer, encoder, vectorAssembler])

학습 준비 과정
- 변환자(transformer)를 데이터셋에 적합(fit)
- StringIndexer: 인덱싱할 고윳값의 수를 알아야 한다.

In [19]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

파이프라인이 준비되면 파이프라인을 사용해서 일관적이고 반복적인 방식으로 모든 데이터를 변환

In [20]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

데이터 캐싱
- 캐싱을 사용하면 중간 변환된 데이터셋의 복사본을 메모리에 저장하므로
- 전체 파이프라인을 재실행하는 것보다 훨씬 빠르게 반복적으로 데이터셋에 접근할 수 있다.

In [21]:
transformedTraining.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

모델 학습
- 머신러닝 모델을 사용하기 위해서는 클래스를 임포트하고 인스턴스를 먼저 생성시켜야 한다.

In [22]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans()\
  .setK(20)\
  .setSeed(1)

스파크에서 머신러닝 모델
1. 학습되지 않은 모델 초기화 시키는 과정: Algorithm
2. 해당 모델을 학습 시키는 과정: AlgorithmModel

In [25]:
kmModel = kmeans.fit(transformedTraining)

모델 학습 완료
- 성과 평가지표에 따라 학습 데이터셋에 대한 비용을 계싼 가능
- 예제. 군집 비용이 높은 편
- 입력 데이터에 대한 전처리와 표준화 작업이 필요하다

In [28]:
transformedTest = fittedPipeline.transform(testDataFrame)

# kmModel.computeCost(transformedTest)
# In PySpark 3.0 and later, computeCost() was removed because it was inefficient.
# Instead, you can use the summary attribute to get the cost.

kmModel.summary.trainingCost

91509143.6661866